In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

%matplotlib inline

# Reading Data

In [2]:
df = pd.read_csv('datasets/Mall_Customers.csv')
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [3]:
# dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
CustomerID                200 non-null int64
Gender                    200 non-null object
Age                       200 non-null int64
Annual Income (k$)        200 non-null int64
Spending Score (1-100)    200 non-null int64
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


# Removing Gender because it is Categorical

In [4]:
df.drop('Gender',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
CustomerID                200 non-null int64
Age                       200 non-null int64
Annual Income (k$)        200 non-null int64
Spending Score (1-100)    200 non-null int64
dtypes: int64(4)
memory usage: 6.3 KB


In [6]:
df.head()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,2,21,15,81
2,3,20,16,6
3,4,23,16,77
4,5,31,17,40


# Clustering Algorithm(K-mediod) 

### Defining Distance metrics

In [7]:
# Euclidian distance calculation
def dist(p1,p2):
    return np.sqrt(np.sum(np.square(p1-p2)))
    

### Set value of K and genratre disimilarity matrix

In [8]:
# value of K (you can change it)
K = 5
# value of D
D = []

# Calculating dissimilarity matrix
for i in range(len(df)):
    dist_lst = []
    for j in range(len(df)):
        dist_lst.append(dist(df.iloc[i],df.iloc[j]))
    D.append(dist_lst)
D = np.matrix(D)




In [9]:
len(df)

200

### Randomly initialize with K random centers 

In [10]:
# Randomly genrate medoids 
medoids = (random.sample(range(len(df)),K))
medoids

[27, 46, 172, 83, 104]

### Assign each point to nearest medoid

In [11]:
P = []  # P contains Cluster Ids [0,K-1]

# iterate through each row of dataframe
for i in range(len(df)):
    
    distances_from_medoids = []
    # iterate through mediods
    for j in medoids:
        distances_from_medoids.append(D[i,j])
    
    P.append(np.argmin(distances_from_medoids))



In [12]:
# setup for 100 iterations
for itr in range(100):
    #print('initall Clustering ' ,P)

    df['Cluster_id'] = P
    
    #list that will contain new medoids
    new_medoids = []
    
    # selecting medoid for each cluster
    for c in range(K):
        
        #select all points belonging to cluster c =  [0,1,2..,K-1]
        cluster_df = df[df['Cluster_id']==c]
        
        # costlist that will contain cost of each point within cluster
        cost_list = []
        
        for i in range(len(cluster_df)):
            cost_i =0
            for j in range(len(cluster_df)):
                
                #index of point i and j in original df to find distance from dissimilarity matrix
                idx_i = (cluster_df.iloc[i]).name
                idx_j = (cluster_df.iloc[j]).name
                cost_i+=D[idx_i,idx_j]
                
            cost_list.append(cost_i)

        # find point with min cost in cost_list, this will be our cluster head
        indx = (cluster_df.iloc[np.argmin(cost_list)]).name
        
        # add new head to medoid list
        new_medoids.append(indx)

    #print('orignal medoids : ', medoids)
    #print('new medoids : ', new_medoids)
    
    #Check if medoids did change (in this case the clustering will be same so break out)
    if np.sum(np.equal(medoids,new_medoids)) == len(medoids):
        #if did not change then break
        #print('SAME')
        break;

    # do clustering with new medoids
    new_P = []
    
    # iterate through each row of df
    for i in range(len(df)):

        distances_from_medoids = []
        
        # iterate through new medoids
        for j in new_medoids:
            
            distances_from_medoids.append(D[i,j])

        new_P.append(np.argmin(distances_from_medoids))

    
    #print('orignal Clustering ' , P)
    #print('new Clustering ',new_P)
    
    #Check if clustering did not change
    if np.sum(np.equal(P,new_P)) == len(P):
        # clustering is same
        #print('SAME')
        break;
    else:
        # clustering changed hence, assign new clustering
        medoids = new_medoids
        P = new_P
    #print()
    
medoids

[21, 42, 168, 77, 122]

# Calculating Average silhouette width

In [13]:
# Average silhouette with
S=0

# iterate through each row in df
for i in range(len(df)):
    
    #calculating a_i
    
    # select all points belonging to that cluster
    a_df = df[df['Cluster_id']==df['Cluster_id'].iloc[i]]
    
    a_i=0
    
    # iterate through all points in that cluster
    for j in range(len(a_df)):
        
        #index of point i and j in original df to find distance from dissimilarity matrix
        idx_i = (df.iloc[i]).name
        idx_j = (a_df.iloc[j]).name
        a_i +=D[idx_i,idx_j]
        
    # a = (distacne from each point in cluster)/(total points in cluster)   
    a_i=(a_i/len(a_df))
    
    # find nearest neighbouring cluster for the point
    lst = []
    # iterate through each cluster j = [0,1,2,..K-1]
    for j in range(len(medoids)):
        
        # if same cluster than append max value so that it doesnot get selected as nearest neighbouring cluster
        if j==df['Cluster_id'].iloc[i]:
            lst.append(2**31)
        # for other cluster find distance from each point in that cluster
        else:
            # find all points belonging to cluster j
            neigh = df[df['Cluster_id']==j]
            c_n=0
            for j in range(len(neigh)):
                #index of point i and j in original df to find distance from dissimilarity matrix
                idx_i = (df.iloc[i]).name
                idx_j = (neigh.iloc[j]).name
                c_n +=D[idx_i,idx_j]
            lst.append(c_n/len(neigh))
            
    #nearest neighbour cluster_id    
    C_neig = np.argmin(lst)
    #print(lst[C_neig])
    
    #calculating b_i
    b_i=lst[C_neig]
    
    #  silhouette width for point i
    S_i = (b_i - a_i)/max(b_i,a_i)
    #print('Si',S_i)
    S+=S_i

# average silhouette width
S = S/len(df)
print("Average silhouette width : ",S)

        

Average silhouette width :  0.3193334254653304


# Save it as .csv file

In [14]:
df['Cluster_id'] = P
filename = f"Cluster_{K}.csv"
df.to_csv(filename)